# Imports

In [ ]:
# face_recognition, cmake, dlib

In [3]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
import face_recognition
import cv2

# Set-Up Labels

In [20]:
labels = pd.read_csv('converted_keras/labels.txt', header = None)

labels = labels.rename(columns = {0:'Names'})
new_index = []
new_labels = []

for x in labels['Names']:
    new_index.append(int(x.split(" ", 1)[0]))  # to ensure it is not appended as a list
    new_labels.append(x.split(" ", 1)[1])
    
prediction_dict = dict(zip(new_index, new_labels))

# Code copied from Google 

In [21]:
# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model (set compile to False)
model = tensorflow.keras.models.load_model('converted_keras/keras_model.h5', compile = False)


In [22]:
# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open('img/test/radcliffe_1.jpg')

#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)

# display the resized image
# image.show()

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)

predicted_name = prediction_dict[prediction.argmax()]

print(prediction)
print(predicted_name)

[[0.17614621 0.73739374 0.08646011]]
Rupert Grint


In [10]:
# This allows it to now return the name of the prediction

# Using Computer Vision

In [54]:
# cap = cv2.VideoCapture(0)

In [55]:
# success, img = cap.read()

In [56]:
# cap.release()

In [57]:
# res = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
# normalized_image_array = (res.astype(np.float32) / 127.0) - 1
# data[0] = normalized_image_array
# prediction = model.predict(data)
# print(prediction)
# print(prediction_dict[2])

[[0.14452581 0.5813541  0.27412015]]
Daniel Radcliffe


In [63]:
# cv2.imshow("Webcam",img)

In [64]:
# cv2.waitKey(1)

In [29]:
cap = cv2.VideoCapture(0)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# while True:
success, img = cap.read()
res = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
normalized_image_array = (res.astype(np.float32) / 127.0) - 1

data[0] = normalized_image_array
prediction = model.predict(data)

predicted_score = round(prediction.max(),2)
predicted_name = prediction_dict[prediction.argmax()]

print(prediction)
print(f'{predicted_name}: {predicted_score}')

# Face Location
face_locations = face_recognition.face_locations(img)

# print("Found {} faces in this photograph.".format(len(face_locations)))
for face_location in face_locations:
    # Print the location of each face in this image
    top, right, bottom, left = face_location

    # We can draw rectangle using OpenCV rectangle method
    cv2.rectangle(img, (left, top), (right, bottom), (255, 255, 1), 3)   # Face Rectangle
    cv2.putText(img,predicted_name, (left+6, bottom+32), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)     # Name Text
    cv2.putText(img,str(predicted_score) + " probability", (left+6, bottom+70), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)  # Probability

#? is it supposed to show img or res?
cv2.imshow("Webcam",img)
cv2.waitKey(1)

[[0.09683815 0.79991466 0.10324719]]
Rupert Grint: 0.800000011920929


-1

In [30]:
cap.release()


In [19]:
empty_frame = pd.DataFrame()
empty_frame.to_csv('attendance_register.csv')

In [16]:
from datetime import datetime

In [12]:
# from https://analyticsindiamag.com/a-complete-guide-on-building-a-face-attendance-system/

def attendance(name):
    with open('Attendance_Register.csv','r+') as f:
        DataList = f.readlines()
        names = []
        for data in DataList:
            ent = data.split(',')
            names.append(ent[0])
        if name not in names:
            curr = datetime.now()
            dt = curr.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dt}')